In [1]:
import numpy as np
import pandas as pd
import random
from ewmaCalculator import ewma
from statsmodels.tsa.seasonal import seasonal_decompose

import matplotlib.pyplot as plt
from matplotlib import rcParams
from cycler import cycler

rcParams['figure.figsize'] = 18, 5
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['axes.prop_cycle'] = cycler(color=['#365977'])
rcParams['lines.linewidth'] = 2.5

#load Data
df = pd.read_csv('RankingsTransposed.csv', index_col='Year', parse_dates=True)
teams = df.keys()
"""
for n in teams:
    randColor = "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    plt.plot(df[n], color = randColor , label = n)
plt.legend()
plt.show() """

'\nfor n in teams:\n    randColor = "#"+\'\'.join([random.choice(\'0123456789ABCDEF\') for j in range(6)])\n    plt.plot(df[n], color = randColor , label = n)\nplt.legend()\nplt.show() '

In [ ]:
def ewma(df, span, alpha):
    predictions = []
    startIndex = 0
    end = span
    while(len(df) > end):
        dataSet = df[startIndex:end]
        movingAverage = 0
        for n in range(span):
            movingAverage += (alpha) * ((1-alpha) ** (span - n - 1))*df[n + startIndex]
          
        
        if (movingAverage + 1) > 0:
            predictions.append(df[end] - movingAverage)
        startIndex += 1
        
        end += 1
    return predictions

In [17]:
span = 5
def getPredictDiff(df, teams, alpha):
    predictions = []
    for n in teams:
        for scores in ewma(df[n][:-3],span,alpha):
            predictions.append(abs(scores))
    return sum(predictions)

In [3]:
IGNORE = """def findBestAlpha(df, teams, step, alpha, prevDiff):
    diff = getPredictDiff(df, teams, alpha)
    print("FIRST",step,alpha,diff)
    diffUp = findBestAlpha(df, teams, step/2, alpha + step, diff)
    diffDown = findBestAlpha(df, teams, step/2, alpha - step, diff)
    if diffUp < diff and diffUp < diffDown:
        a = findBestAlpha(df, teams, step, alpha + step, diff)
        print(step, alpha)
        return a
    elif diffDown < diff:
        b = findBestAlpha(df, teams, step, alpha - step, diff)
        print(step, alpha)
        return b
    else:
        c =  findBestAlpha(df, teams, step/2, alpha, diff)
        print(step, alpha)
        return c """

In [18]:
#Tries out a bunch of different alphas
minDiff = 2**31 -1
bestAlpha = -1
for n in range(1,100):
    alpha = float(n)/float(100)
    diff = getPredictDiff(df, teams, alpha)
    if diff < minDiff:
        minDiff = diff
        bestAlpha = alpha
alpha = bestAlpha - 0.01
for n in range(1,100):
    alpha += 0.0002
    diff = getPredictDiff(df, teams, alpha)
    if diff < minDiff:
        minDiff = diff
        bestAlpha = alpha
bestAlpha

0.6817999999999987

In [9]:
def getTestDiff(df, teams, alpha):
    predictions = []
    for n in teams:
        for scores in ewma(df[n][:-3],span,alpha):
            predictions.append(abs(scores))
    return sum(predictions)/float(len(predictions))

In [14]:
#Gets average absolute difference for teams using the best alpha from testing
getTestDiff(df,teams,bestAlpha)

44.065095076650195

In [16]:
#Gets average absolute difference just using previous year's ranking
getTestDiff(df,teams,1)

45.83304610081861

45.83304610081861